# Reading Document

In [34]:
from langchain.document_loaders import TextLoader

In [35]:
loader = TextLoader("../data/nvidia.txt")
data = loader.load()

In [36]:
from langchain.document_loaders import CSVLoader

In [37]:
loader_csv = CSVLoader("../data/movies.csv", source_column="title")
datacsv = loader_csv.load()
len(datacsv)

9

In [ ]:
datacsv

In [ ]:
!pip install unstructured libmagic python-magic

In [40]:
from langchain.document_loaders import UnstructuredURLLoader

In [41]:
url_loader = UnstructuredURLLoader(urls = [
        "https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html",
        "https://www.moneycontrol.com/news/business/markets/market-corrects-post-rbi-ups-inflation-forecast-icrr-bet-on-these-top-10-rate-sensitive-stocks-ideas-11142611.html"
    ])

In [42]:
dataurl = url_loader.load()
len(dataurl)

2

In [43]:
dataurl[0].page_content[0:100]

'English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Pro'

# Test Splitter

In [44]:
# Taking some random text from wikipedia

text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. 
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. 
Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg. 
Kip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. 
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles. 
Interstellar uses extensive practical and miniature effects, and the company Double Negative created additional digital effects.

Interstellar premiered in Los Angeles on October 26, 2014. In the United States, it was first released on film stock, expanding to venues using digital projectors. The film received generally positive reviews from critics and grossed over $677 million worldwide ($715 million after subsequent re-releases), making it the tenth-highest-grossing film of 2014. 
It has been praised by astronomers for its scientific accuracy and portrayal of theoretical astrophysics.[5][6][7] Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""



In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [46]:
r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 200,  # size of each chunk created
    chunk_overlap  = 0,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
)

In [47]:
chunks = r_splitter.split_text(text)

for chunk in chunks:
    print(len(chunk))


105
120
199
10
181
197
198
8
128
191
165
198
54


# Vectory DB

In [4]:
import pandas as pd

pd.set_option('display.max_colwidth', 100)

In [5]:
df = pd.read_csv('../data/sample_text.csv')

In [6]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [8]:
from sentence_transformers import SentenceTransformer

In [9]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [13]:
vectors

array([[-0.0024738 ,  0.03626736, -0.05290458, ..., -0.09152357,
        -0.03970012, -0.04330489],
       [-0.03357267,  0.0098052 , -0.03250134, ..., -0.05165479,
         0.02245886, -0.0315619 ],
       [-0.01865334, -0.04051305, -0.01235391, ...,  0.00610582,
        -0.07179652,  0.02773848],
       ...,
       [-0.00066462,  0.04252121, -0.05645505, ...,  0.01315469,
        -0.03183562, -0.04357658],
       [-0.03317151,  0.0325246 , -0.0248484 , ...,  0.01174418,
         0.05747117,  0.00571026],
       [-0.00166398,  0.00413838, -0.04597079, ...,  0.02008529,
         0.05656247, -0.00161593]], dtype=float32)

In [10]:
vectors.shape

(8, 768)

In [12]:
dim = vectors.shape[1]
dim

768

In [15]:
import faiss

In [17]:
index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0xffff6331de10> >

In [19]:
index.add(vectors)

In [21]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape


(768,)

In [23]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [25]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844838, 1.4039093]], dtype=float32)

In [26]:
I

array([[3, 2]])

In [27]:
df.loc[[3,2]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion
